In [24]:
!det experiment create const.yaml .

Preparing files to send to master... 153.5KB and 15 files
Created experiment 239


In [19]:
import pandas as pd
df = pd.read_csv('diabetes.csv')

In [23]:
df.to_numpy()

array([[  6.   , 148.   ,  72.   , ...,   0.627,  50.   ,   1.   ],
       [  1.   ,  85.   ,  66.   , ...,   0.351,  31.   ,   0.   ],
       [  8.   , 183.   ,  64.   , ...,   0.672,  32.   ,   1.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,   0.245,  30.   ,   0.   ],
       [  1.   , 126.   ,  60.   , ...,   0.349,  47.   ,   1.   ],
       [  1.   ,  93.   ,  70.   , ...,   0.315,  23.   ,   0.   ]])

In [2]:
from determined.experimental import Determined
import warnings
warnings.filterwarnings("ignore")
checkpoint = Determined().get_experiment(210).top_checkpoint()
model = checkpoint.load().model

NotFoundException: {"error":{"code":5,"reason":"NotFound","error":"experiment not found: 210"}}


In [76]:
from PIL import Image
from torch.autograd import Variable
import torchvision.transforms as transforms
import torch
import glob

In [123]:
transformer=transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])
])

In [124]:
def prediction(img_path,transformer):
    image=Image.open(img_path)
    image_tensor=transformer(image).float()    
    image_tensor=image_tensor.unsqueeze_(0)
    if torch.cuda.is_available():
        image_tensor.cuda()
    # input=Variable(image_tensor.cuda())
    input=Variable(image_tensor)
    # output = ""
    # with torch.no_grad():
    output = model(input)
    # index = np.argmax(output.cuda())[0].cpu().numpy()
    # index=output.data.numpy().argmax().cpu().numpy()
    index = output.cpu().data.numpy().argmax()
    classes = ['buildings', 'forest', 'glacier', 'mountain', 'sea','street']
    pred=classes[index]   
    return pred

In [125]:
pred_path='/run/determined/workdir/demo/scene_detectionDemo/test'
images_path=glob.glob(pred_path+'/*.jpg')

In [126]:
images_path

['/run/determined/workdir/demo/scene_detectionDemo/test/20216.jpg',
 '/run/determined/workdir/demo/scene_detectionDemo/test/20212.jpg']

In [127]:
pred_dict={}

for i in images_path:
    pred_dict[i[i.rfind('/')+1:]]=prediction(i,transformer)

In [128]:
pred_dict

{'20216.jpg': 'street', '20212.jpg': 'sea'}